# Access Azure resources from an online endpoint with a system-assigned managed identity

The `azure.mgmt.storage` package is required for this example. 

Install it with the following code:

In [ ]:
# #TODO: 
# %pip install --pre azure-mgmt-storage
# %pip install --pre azure-mgmt-authorization

## 1. Configure variables

### 1.1 Assign variables for the workspace and deployment

In [ ]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace_name = "<AML_WORKSPACE_NAME>"

endpoint_name = "<ENDPOINT_NAME>"

In [2]:
# TODO: Delete

subscription_id = "6fe1c377-b645-4e8e-b588-52e57cc856b2"
resource_group = "role-creation-sample"
workspace_name = "test-workspace"

endpoint_name = "rolestest4"

### 1.2 Specify the storage details and file to access
A text file is located in the `managed-identities` folder called `hello.txt`. 

In [3]:
storage_account_name = "<STORAGE_ACCOUNT_NAME>"
storage_container_name = "<CONTAINER_TO_ACCESS>"
file_name = "<FILE_TO_ACCESS>"

In [4]:
# TODO: Delete

storage_account_name = "rolestest2"
storage_container_name = "blob1"
file_name = "hello.txt"

### 1.3 Retrieve the workspace location:

In [5]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, AzureCliCredential
from azure.ai.ml.entities import (
    ManagedOnlineDeployment,
    ManagedOnlineEndpoint,
    Model,
    CodeConfiguration,
    Environment,
)

credential = AzureCliCredential()
ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)

workspace_location = ml_client.workspaces.get(workspace_name).location

## 2. Configure deployment

### 2.1 Define an endpoint configuration

In [6]:
endpoint = ManagedOnlineEndpoint(name=endpoint_name, auth_mode="key")

### 2.2 Define a deployment configuration

In [9]:
deployment = ManagedOnlineDeployment(
    name="purple", # TODO
    endpoint_name=endpoint_name,
    model=Model(path="../../model-1/model/"),
    code_configuration=CodeConfiguration(
        code="../../model-1/onlinescoring/", scoring_script="score_managedidentity.py"
    ),
    environment=Environment(
        conda_file="../../model-1/environment/conda.yml",
        image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210727.v1",
    ),
    instance_type="Standard_DS2_v2",
    instance_count=1,
    environment_variables={
        "STORAGE_ACCOUNT_NAME": storage_account_name,
        "STORAGE_CONTAINER_NAME": storage_container_name,
        "FILE_NAME": file_name,
    },
)

..

## 3. Create the managed identity

With a system-assigned managed identity, Azure creates a managed identity on your behalf.

## 4. Create storage account and container

### 4.1 Get a handle to the `StorageManagementClient`

In [ ]:
from azure.mgmt.storage import StorageManagementClient
from azure.storage.blob import ContainerClient
from azure.mgmt.storage.models import Sku, StorageAccountCreateParameters, BlobContainer

credential = AzureCliCredential()
storage_client = StorageManagementClient(
    credential=credential, subscription_id=subscription_id
)

### 4.2 Define and create the storage account

In [ ]:
storage_account_parameters = StorageAccountCreateParameters(
    sku=Sku(name="Standard_LRS"), kind="Storage", location=workspace_location
)

poller = storage_client.storage_accounts.begin_create(
    resource_group_name=resource_group,
    account_name=storage_account_name,
    parameters=storage_account_parameters,
)
poller.wait()

storage_account = poller.result()

### 4.3 Create a blob container

In [ ]:
blob_container = storage_client.blob_containers.create(
    resource_group_name=resource_group,
    account_name=storage_account_name,
    container_name=storage_container_name,
    blob_container=BlobContainer(),
)

### 4.4 Get the storage account key and create a `ContainerClient`

In [ ]:
res = storage_client.storage_accounts.list_keys(
    resource_group_name=resource_group,
    account_name=storage_account_name,
)
key = res.keys[0].value

container_client = ContainerClient(
    account_url=storage_account.primary_endpoints.blob,
    container_name=storage_container_name,
    credential=key,
)

### 4.5 Upload a blob to the container

In [ ]:
# TODO: 
file_path = "hello.txt"
with open(file_path, "rb") as f:
    container_client.upload_blob(name=file_name, data=f.read())

## 5. Create an online endpoint

In [ ]:
# #TODO: 
# endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint)

In [ ]:
# TODO: 
endpoint = ml_client.online_endpoints.get(endpoint_name)

In [ ]:
endpoint.identity.as_dict()

## 6. Give access permission to the managed identity

### 6.1 Get a handle to the AuthorizationManagementClient

In [ ]:
from azure.mgmt.authorization import AuthorizationManagementClient
from azure.mgmt.authorization.v2020_10_01_preview.models import (
    RoleAssignment,
    RoleAssignmentCreateParameters,
)
from azure.mgmt.authorization.v2018_01_01_preview.models import (
    RoleDefinition
)
import uuid

role_definition_client = AuthorizationManagementClient(
    credential=credential, subscription_id=subscription_id,
    api_version="2018-01-01-preview"
)

role_assignment_client = AuthorizationManagementClient(
    credential=credential, subscription_id=subscription_id,
    api_version="2020-10-01-preview"
)


### 6.2 Get System-assigned identity details

In [ ]:
endpoint = ml_client.online_endpoints.get(endpoint_name)
system_principal_id = endpoint.identity.principal_id

### 6.3 Give permission to the user storage account

In [ ]:
role_name = "Storage Blob Data Reader"
scope = storage_account.id

role_defs = role_definition_client.role_definitions.list(scope=scope)
role_def = next((r for r in role_defs if r.role_name == role_name))

role_assignment_client.role_assignments.create(
    scope=scope,
    role_assignment_name=str(uuid.uuid4()),
    parameters=RoleAssignmentCreateParameters(
        role_definition_id=role_def.id, principal_id=system_principal_id
    )
)

## 7. Create a deployment with your configuration

### 7.1 Create the deployment

In [10]:
deployment = ml_client.online_deployments.begin_create_or_update(deployment)

Check: endpoint rolestest4 exists
Uploading onlinescoring (0.0 MBs): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3979/

......................................................................................

Done (7m 27s)


### 7.2 Check the status of the deployment

In [16]:
deployment = ml_client.online_deployments.get(endpoint_name=endpoint_name, name="purple") # TODO

### 7.3 Get the deployment logs

In [17]:
ml_client.online_deployments.get_logs(deployment.name, deployment.endpoint_name, 1000)

"Instance status:\nSystemSetup: Succeeded\nUserContainerImagePull: Succeeded\nModelDownload: Succeeded\nUserContainerStart: Succeeded\n\nContainer events:\nKind: Pod, Name: Pulling, Type: Normal, Time: 2022-09-14T22:41:58.242049Z, Message: Start pulling container image\nKind: Pod, Name: Downloading, Type: Normal, Time: 2022-09-14T22:41:59.003268Z, Message: Start downloading models\nKind: Pod, Name: Pulled, Type: Normal, Time: 2022-09-14T22:42:51.467564Z, Message: Container image is pulled successfully\nKind: Pod, Name: Downloaded, Type: Normal, Time: 2022-09-14T22:42:51.467564Z, Message: Models are downloaded successfully\nKind: Pod, Name: Created, Type: Normal, Time: 2022-09-14T22:42:52.016165Z, Message: Created container inference-server\nKind: Pod, Name: Started, Type: Normal, Time: 2022-09-14T22:42:52.146802Z, Message: Started container inference-server\nKind: Pod, Name: ContainerReady, Type: Normal, Time: 2022-09-14T22:43:12.727329156Z, Message: Container is ready\n\nContainer log

### 7.4 Set traffic to 100% for deployment

In [ ]:
endpoint.traffic = {str(deployment.name): 100}
ml_client.begin_create_or_update(endpoint)

## 8. Confirm your endpoint deployed successfully

In [18]:
sample_data = "../../model-1/sample-request.json"
ml_client.online_endpoints.invoke(endpoint_name=endpoint_name, request_file=sample_data)

Exception: No valid deployments to route to. Please check that the endpoint has at least one deployment with positive weight values or use a deployment specific header to route. [https://docs.microsoft.com/en-us/azure/machine-learning/how-to-safely-rollout-managed-endpoints#deploy-a-new-model-but-send-it-no-traffic-yet]

## 9. Delete the endpoint and storage account

### 9.1 Delete the endpoint

In [ ]:
ml_client.online_endpoints.begin_delete(endpoint_name)

### 9.2 Delete the storage account

In [ ]:
storage_client.storage_accounts.delete(
    resource_group_name=resource_group, account_name=storage_account_name
)